In [1]:
import pandas as pd
import os
import json
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from imblearn.over_sampling import SMOTE

# Carregamento dos dados PPG
print(f"\n[Info] Centralizando dados dos participantes...")
base_path = r'C:\Users\ferna\Documents\Unisinos\Cadeiras\TCC 2\Dataset utilizado\GalaxyPPG\Dataset'

# Lista manual com os IDs dos participantes escolhidos
selected_ids = ['P11', 'P12', 'P13', 'P14', 'P15', 'P17', 'P18', 'P19', 'P20']

# Lista para armazenar os DataFrames
dataframes = []

# Iterar sobre os IDs e carrega cada arquivo
for participant_id in selected_ids:
    ppg_path = os.path.join(base_path, participant_id, 'GalaxyWatch', 'PPG.csv')
    
    if os.path.exists(ppg_path):
        df = pd.read_csv(ppg_path)
        df['participant_id'] = participant_id  # Adiciona a origem dos dados
        dataframes.append(df)
    else:
        print(f"[Aviso] Arquivo não encontrado: {ppg_path}")

# Concatenar todos os dados em um único DataFrame
df_centralizado = pd.concat(dataframes, ignore_index=True)

# Exibir resultado
print(f"[Info] Dados centralizados com sucesso. Total de registros: {len(df_centralizado)}!")
print(df_centralizado.head())

ppg_signal = df_centralizado['ppg'].values     # Extrai o sinal PPG (fotopletismografia)
ppg_ts = df_centralizado['timestamp'].values   # Extrai os timestamps (não usado diretamente)
fs = 25  # Frequência de amostragem do sinal (25 Hz)

# Filtragem do sinal PPG
print(f"\n[Info] Aplicando filtro passa-baixa Butterworth para remover ruído de alta frequência dos sinais PPG...")

def butter_lowpass(data, cutoff=5, fs=25, order=3):
    nyq = 0.5 * fs  # Frequência de Nyquist
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)  # Filtro passa-baixa aplicado em modo zero-phase (sem atraso)
    return y

ppg_filtered = butter_lowpass(ppg_signal, cutoff=5, fs=fs, order=3)

# Função para extrair features HRV a partir dos picos detectados
def extract_hrv_features_from_peaks(peaks, fs):
    try:
        if len(peaks) < 3:
            raise ValueError("Poucos picos detectados para análise HRV")

        # Calcular os intervalos entre batimentos (IBI) em segundos
        ibis = np.diff(peaks) / fs
        ibis_ms = ibis * 1000  # Converte para milissegundos

        # Cálculo das métricas HRV
        rmssd = np.sqrt(np.mean(np.square(np.diff(ibis_ms))))       # Raiz da média dos quadrados das diferenças consecutivas
        sdnn = np.std(ibis_ms)                                       # Desvio padrão dos intervalos
        pnn50 = np.sum(np.abs(np.diff(ibis_ms)) > 50) / len(ibis_ms) # Proporção de diferenças maiores que 50ms
        mean_ibi = np.mean(ibis_ms)                                  # Média dos intervalos
        iqr_ibi = np.percentile(ibis_ms, 75) - np.percentile(ibis_ms, 25)  # Intervalo interquartil

        return [rmssd, sdnn, pnn50, mean_ibi, iqr_ibi]
    except Exception as e:
        print(f"Erro no cálculo HRV: {e}")
        return [np.nan] * 5

# Extração das features com janela móvel
window_size = 30 * fs  # Janela de 30 segundos
step_size = 10 * fs    # Passo de 10 segundos (sobreposição de 20 segundos)

features = []
labels = []

print(f"\n[Info] Extraindo features HRV...")

print(f"\n[Info] Definindo rótulo de anormalidade cardíaca...")

for start in range(0, len(ppg_filtered) - window_size, step_size):
    segment = ppg_filtered[start:start + window_size]

    # Detectar picos no segmento com distância mínima de 0.5s entre eles (ajustável)
    peaks, _ = find_peaks(segment, distance=int(fs * 0.5))

    # Ignorar segmentos com poucos picos detectados (não confiável para análise)
    if len(peaks) < 3:
        continue

    # Extrair as métricas HRV a partir dos picos detectados
    hrv = extract_hrv_features_from_peaks(peaks, fs)

    # Ignorar segmentos com métricas inválidas (NaN)
    if np.any(np.isnan(hrv)):
        continue

    rmssd, sdnn = hrv[0], hrv[1]

    # Define o label binário (0 = normal, 1 = possível anormalidade)
    label = int((rmssd < 20) or (sdnn < 50))

    features.append(hrv)
    labels.append(label)

# Converter listas para arrays NumPy
X = np.array(features)
y = np.array(labels)

# Normalização e divisão treino-validação-teste
print(f"\n[Info] Normalizando as features...")
scaler = StandardScaler()
Xs = scaler.fit_transform(X)  # Normaliza as features para média 0 e desvio padrão 1

print(f"\n[Info] Separando dados em treino (60%) e teste (40%)...")
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.4, random_state=42, stratify=y)

print(f"\n[Info] Aplicando SMOTE apenas no conjunto de treino...")
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

print(f"[Info] Após SMOTE - Classe 0: {sum(y_train == 0)}, Classe 1: {sum(y_train == 1)}")

# Treinamento do modelo de Regressão Logística
CLASSES = np.array([0, 1])

# Criar modelo
model = SGDClassifier(
    loss='hinge',  # Função de perda "hinge", que implementa o classificador linear de margem máxima usado no SVM
    penalty='elasticnet',   # Penalização Elastic Net: combinação de L1 (sparsidade) e L2 (regularização de peso)
    l1_ratio=0.15,          # Proporção do L1 na penalização Elastic Net (0 = só L2, 1 = só L1); aqui 15% L1 e 85% L2
    max_iter=1000,          # Número máximo de épocas (passadas pelos dados) por chamada ao fit/partial_fit; 
    learning_rate='optimal',# Estratégia de taxa de aprendizado: 'optimal' usa uma taxa adaptativa baseada no número de iterações
    eta0=0.01,              # Taxa de aprendizado inicial (usada se o learning_rate permitir, aqui influencia o cálculo adaptativo)
    alpha=1e-4,             # Fator de regularização global (quanto maior, mais forte a regularização e menos ajuste aos dados)
    average=True,           # Faz média dos coeficientes ao longo das iterações, geralmente melhora estabilidade e generalização
    random_state=42         # Semente para reprodutibilidade dos resultados
)

# Treinamento
print(f"\n[Info] Treinando modelo com {len(X_train)} exemplos...")
model.fit(X_train, y_train)

# Salvar modelo final em arquivo JSON
MODEL_FILE = 'Modelo_Centralizado_SVM.json'
print(f"\n[Nuvem] Salvando modelo final...")
final_model_data = {
    "class": model.__class__.__name__,
    "params": model.get_params(),
    "coef": model.coef_.tolist(),
    "intercept": model.intercept_.tolist()
}
with open(MODEL_FILE, 'w') as f:
    json.dump(final_model_data, f)
print(f"[Nuvem] Modelo salvo em '{MODEL_FILE}'.")

# Avaliação do modelo
print(f"\n[Info] Avaliando modelo com dados de treino...")
y_train_scores = model.decision_function(X_train)
y_train_prob = (1 / (1 + np.exp(-y_train_scores)))
y_pred = (y_train_prob >= 0.5).astype(int)

print(classification_report(y_train, y_pred))
print(f"Acurácia final do modelo: {accuracy_score(y_train, y_pred):.4f}")

print("\nMatriz de Confusão:")
print(confusion_matrix(y_train, y_pred))

print(f"\n[Info] Avaliando modelo com dados de teste...")

# Previsões com threshold padrão 0.5
y_test_scores = model.decision_function(X_test)
y_test_prob = (1 / (1 + np.exp(-y_test_scores)))
y_pred = (y_test_prob >= 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(f"Acurácia final do modelo: {accuracy_score(y_test, y_pred):.4f}")

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


[Info] Centralizando dados dos participantes...
[Info] Dados centralizados com sucesso. Total de registros: 864000!
    dataReceived      timestamp      ppg  status participant_id
0  1711437352342  1711437340286  2097152       0            P11
1  1711437352342  1711437340327  2097152       0            P11
2  1711437352342  1711437340369  2462981       0            P11
3  1711437352342  1711437340408  2462958       0            P11
4  1711437352342  1711437340447  2462983       0            P11

[Info] Aplicando filtro passa-baixa Butterworth para remover ruído de alta frequência dos sinais PPG...

[Info] Extraindo features HRV...

[Info] Definindo rótulo de anormalidade cardíaca...

[Info] Normalizando as features...

[Info] Separando dados em treino (60%) e teste (40%)...

[Info] Aplicando SMOTE apenas no conjunto de treino...
[Info] Após SMOTE - Classe 0: 1487, Classe 1: 1487

[Info] Treinando modelo com 2974 exemplos...

[Nuvem] Salvando modelo final...
[Nuvem] Modelo salvo em 'Mo

C:\Users\ferna\AppData\Local\Temp\ipykernel_22480\4153984810.py:185: RuntimeWarning: overflow encountered in exp
  y_train_prob = (1 / (1 + np.exp(-y_train_scores)))
